In [1]:
import matplotlib.pyplot as plt
from mxESP32Debugger.debugger import Dbg
import time
import toml
import numpy as np                                                                                                      
from loguru import logger

In [2]:
with open("gc6153.toml", 'r', encoding='utf-8') as f:
    config = toml.load(f)
    
init_reg_list = config['init']['reg_list']
mode_reg_list = config['mode']['reg_list']

In [3]:
dev = Dbg()
dev.usb_config(False)
dev.power_init()

2025-03-19 13:46:05.605 | INFO     | mxESP32Debugger.debugger:connect:249 - Using serial port to connect. Port: COM4
2025-03-19 13:46:05.607 | INFO     | mxESP32Debugger.debugger:connect:254 - Using ESP32-S3R8.
2025-03-19 13:46:05.608 | INFO     | mxESP32Debugger.debugger:connect:255 - Embedded software version: v1.0
2025-03-19 13:46:05.608 | INFO     | mxESP32Debugger.debugger:connect:256 - Library version: v241210.0.rc3.
2025-03-19 13:46:05.631 | INFO     | mxESP32Debugger.debugger:get_extboard_version:1718 - Extension board version: v0.2.1


True

In [4]:
dev.i2c_config(sda_pullup=False, scl_pullup=False, freq=400_000)

(True, None)

In [5]:
dev.power_control("I2C", "3v3")
dev.power_control("SPI", "3v3")

True

In [6]:
pdn_pin = 39
mclk_en_pin = 40
dev.gpio_config(pin=pdn_pin, mode=0x02, pull_up=True, pull_down=False)
dev.gpio_config(pin=mclk_en_pin, mode=0x02, pull_up=True, pull_down=False)
dev.gpio_write_read(pin=mclk_en_pin, level=1)

(True, None)

In [7]:
time.sleep(100/1000)
dev.gpio_write_read(pin=pdn_pin, level=1)
time.sleep(10/1000)
dev.gpio_write_read(pin=pdn_pin, level=0)
time.sleep(10/1000)
dev.gpio_write_read(pin=pdn_pin, level=1)
time.sleep(10/1000)
dev.gpio_write_read(pin=pdn_pin, level=0)

(True, None)

In [8]:
ret, chipid = dev.i2c_write_read(0x40, [0xf0], 2)
assert ret, "I2C read chipid failed"
logger.info("chipid: 0x{:04X}", (chipid[0] << 8) | chipid[1])

2025-03-19 13:46:05.994 | INFO     | __main__:<module>:3 - chipid: 0x6153


In [9]:
for reg in init_reg_list:
    ret, _ = dev.i2c_write_read(slave_id=0x40, write_list=reg, read_length=0)
    assert ret, "I2C write init reg list error"
for reg in mode_reg_list:
    ret, _ = dev.i2c_write_read(slave_id=0x40, write_list=reg, read_length=0)
    assert ret, "I2C write mode reg list error"
    

In [10]:
read_size = 320 * 240 * 2 + (320 * 12) + (9 + 4)
logger.info(f"read_size: {read_size}")

aligned_size = 128
max_transfer_size = aligned_size - read_size % aligned_size + read_size
logger.info(f"max_transfer_size: {max_transfer_size}")

2025-03-19 13:46:07.405 | INFO     | __main__:<module>:2 - read_size: 157453
2025-03-19 13:46:07.406 | INFO     | __main__:<module>:6 - max_transfer_size: 157568


In [11]:
# configure spi as slave mode
dev.spi_config(miso_io_num=12, mosi_io_num=13, sclk_io_num=14, cs_io_num=15, mode=3, master_slave_mode=1, max_transfer_sz=max_transfer_size)

(True, None)

In [13]:
ret, data = dev.spi_write_read([], read_size, timeout=5, debug_mode=False)
assert ret, "Read image failed"

with open("data_raw.txt", 'w', encoding='utf-8') as f:
    
    for i in range(len(data)):
        f.write(f"{data[i]:02X} ")

TimeoutError: Data read timeout. No data received.

In [ ]:
with open("data_raw_2.txt", 'r', encoding='utf-8') as f:
    # read line
    lines = f.readlines()

In [ ]:
line_data = [] # save int data
for line in lines:
    # print(len(line))
    line_temp = line.replace('\n', '').split(' ')
    # print(line_temp)
    
    single_line_data = []
    for i in range(len(line_temp)):
        if line_temp[i] != '':
            single_line_data.append(int(line_temp[i], 16))
    line_data.append(single_line_data)
    
# check if all line_data[:4] if ff ff ff 02
for i in range(4):
    if line_data[i][0] != 0xFF or line_data[i][1] != 0xFF or line_data[i][2] != 0xFF or line_data[i][3] != 0x02:
        logger.error(f"line {i} is not start with ff ff ff 02")
        break

sorted_data = line_data

line_data_image = [sorted_data[i][12:] for i in range(len(sorted_data))]

line_data_image_check = []
for i, line in enumerate(line_data_image):
    if len(line) > 480:
        line_data_image_check.append(line[:480])
    else:
        line_data_image_check.append(line)

In [ ]:
for i in range(len(line_data_image_check)):
    
    if len(line_data_image_check[i]) !=  240 * 2:
        logger.error(f"line {i} data length is not 320*240*2")
        break

In [ ]:
np_data = np.array(line_data_image_check)

In [ ]:
def rgb565_to_rgb(rgb565):
    """
    将 RGB565 格式转换为 RGB（8-bit）格式。
    
    参数:
    rgb565 -- 一个包含 RGB565 数据的 numpy 数组（uint16 类型）。
    
    返回:
    一个包含 RGB 格式数据的 numpy 数组，数据类型为 uint8。
    """
    # 提取 R, G, B 分量
    r = ((rgb565 >> 11) & 0x1F) * (255 // 31)
    g = ((rgb565 >> 5) & 0x3F) * (255 // 63)
    b = (rgb565 & 0x1F) * (255 // 31)
    
    # 合并成 RGB 图像
    rgb_image = np.stack((r, g, b), axis=-1).astype(np.uint8)
    
    return rgb_image

In [ ]:
image_data = []
for line in np_data:
    line_data = []
    for i in range(0, len(line), 2):
        line_data.append(line[i] << 8 | line[i+1])
    image_data.append(line_data)
image_data = np.array(image_data).reshape(320, -1)

plt.imshow(image_data, cmap='gray', aspect='auto')
plt.axis('off')
plt.show()

image_data = rgb565_to_rgb(image_data)
plt.imshow(image_data, cmap='gray', aspect='auto')
plt.axis('off')
plt.show()